<h2>Feature Creation</h2>

In [2]:
import time
import os
import pandas as pd
import sys
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import newspaper
import ast

from IPython.display import clear_output

In [3]:
comments = pd.read_csv('./train_comments.csv',low_memory=False)
comments[['body']] = comments[['body']].astype(str)
comments['keywords'] = comments.keywords.apply(ast.literal_eval)
comments.drop_duplicates(inplace=True,subset='id')
comments.reset_index(drop=True)
print comments.shape
comments.head()

(275686, 25)


,author,body,body_html,controversiality,created,created_utc,distinguished,downs,edited,gilded,...,parent_id,replies,score,subreddit,ups,pid,tokens,comment_length,n_tokens,keywords
0,SirT6,The title sort of misses the point of the stud...,"&lt;div class=""md""&gt;&lt;p&gt;The title sort ...",0,1447279564,1447250764,NaN,0,False,1,...,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",1359,science,1359,3se6lu,"Counter({'alga': 5, 'cancer': 4, 'cell': 4, 'd...",869,52,"{u'toxinalgae': 1.00993377483, u'cancer': 1.03..."
1,DrBiochemistry,Just want to point out that until I see a deli...,"&lt;div class=""md""&gt;&lt;p&gt;Just want to po...",0,1447277409,1447248609,NaN,0,False,0,...,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",3209,science,3209,3se6lu,"Counter({'kill': 2, 'deliveri': 2, 'cancer': 1...",307,30,"{u'survives': 1.02941176471, u'thing': 1.02941..."
2,Frogblood,It's an interesting idea but the in vitro and ...,"&lt;div class=""md""&gt;&lt;p&gt;It&amp;#39;s an...",0,1447276156,1447247356,NaN,0,False,0,...,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",133,science,133,3se6lu,"Counter({'idea': 2, 'target': 2, 'overexcit': ...",432,39,"{u'tumour': 1.02173913043, u'targeting': 1.043..."
3,mijn_ikke,Just waiting until somebody smarter than me co...,"&lt;div class=""md""&gt;&lt;p&gt;Just waiting un...",0,1447275611,1447246811,NaN,0,1447248944.0,1,...,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",773,science,773,3se6lu,"Counter({'thank': 1, 'gold': 1, 'point': 1, 'e...",163,12,"{u'somebody': 1.05172413793, u'gold': 1.051724..."
4,awhitt8,Yes the title is sensationalized.\n\n&gt;The m...,"&lt;div class=""md""&gt;&lt;p&gt;Yes the title i...",0,1447284967,1447256167,NaN,0,1447259263.0,0,...,t3_3se6lu,"{u'kind': u'Listing', u'data': {u'modhash': No...",16,science,16,3se6lu,"Counter({'drug': 5, 'deliveri': 4, 'materi': 3...",1447,104,"{u'silicon': 1.01530612245, u'tissue': 1.01530..."


In [4]:
articles = pd.read_csv('./train_articles.csv',low_memory=False)
articles[['text','summary']] = articles[['text','summary']].astype(str)
articles.drop_duplicates(inplace=True,subset = 'url')
articles['keywords'] = articles.keywords.apply(ast.literal_eval)
articles.reset_index(drop=True,inplace=True)
print articles.shape
articles.head()

(11420, 28)


,authors,keywords,publish_date,summary,text,url,author,created_utc,domain,downs,...,permalink,score,selftext,subreddit,thumbnail,title,ups,tokens,article_len,n_tokens
0,"[Derek Keats Flickr, Hannah Osborne, Marc Cirera]","{u'cancer': 1.0261627907, u'drugs': 1.03052325...",2015-11-10 16:00:00+00:00,Algae has been genetically engineered to kill ...,Algae has been genetically engineered to kill ...,http://www.ibtimes.co.uk/algae-genetically-eng...,the_phet,1447239366,ibtimes.co.uk,0,...,/r/science/comments/3se6lu/algae_has_been_gene...,6705,NaN,science,http://b.thumbs.redditmedia.com/y1CGKgl69hKw-s...,Algae has been genetically engineered to kill ...,6705,"Counter({'alga': 11, 'drug': 9, 'cell': 8, 'ca...",2352,2352
1,[],"{u'diamond': 1.04316546763, u'laser': 1.008633...",None,"If Q-carbon is harder than diamond, why would ...","This is a collection of 0.02, 0.03 and 0.04 ca...",http://phys.org/news/2015-11-phase-carbon-diam...,skoalbrother,1448903226,phys.org,0,...,/r/science/comments/3uvg0o/researchers_find_ne...,6777,NaN,science,http://b.thumbs.redditmedia.com/hZrhEdBoJp22oE...,"Researchers find new phase of carbon, make dia...",6777,"Counter({'diamond': 21, 'qcarbon': 15, 'carbon...",4626,4626
2,[Bjorn Carey],"{u'obfuscation': 1.01181102362, u'fraud': 1.01...",2015-11-16 00:00:00,Stanford researchers uncover patterns in how s...,Stanford researchers uncover patterns in how s...,http://news.stanford.edu/news/2015/november/fr...,godsenfrik,1447707142,news.stanford.edu,0,...,/r/science/comments/3t2exx/when_scientists_fal...,6259,NaN,science,http://b.thumbs.redditmedia.com/FQPskQP8EejVh9...,"When scientists falsify data, they try to cove...",6259,"Counter({'paper': 13, 'research': 11, 'scienti...",4253,4253
3,[],"{u'oil': 1.05142857143, u'microbes': 1.0114285...",None,And that leads to more questions about where m...,"Samantha Joye, a professor of marine sciences ...",http://phys.org/news/2015-11-dispersants-oil-d...,avogadros_number,1447107697,phys.org,0,...,/r/science/comments/3s6xe6/dispersants_did_not...,6146,NaN,science,http://b.thumbs.redditmedia.com/mFQzb4d2QNiyaE...,Dispersants did not help oil degrade in BP spi...,6146,"Counter({'oil': 18, 'dispers': 12, 'joy': 11, ...",3330,3330
4,[More From],"{u'star': 1.02124645892, u'forming': 1.0106232...",2015-11-18 06:00:00,"Together, the two observations allowed the sci...",Shares Share\n\nTweet\n\nE-mail\n\n​Of the tho...,http://www.popularmechanics.com/space/deep-spa...,Letmeirkyou,1447869624,popularmechanics.com,0,...,/r/science/comments/3tbkv6/scientists_have_dis...,6020,NaN,science,http://b.thumbs.redditmedia.com/spD7SnbKlAEY1y...,Scientists have discovered an exoplanet still ...,6020,"Counter({'planet': 22, 'star': 12, 'sallum': 1...",4574,4574


In [5]:
#Create Comment Features
########################
#Tokenized Comments

import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
stopset = set(stopwords.words('english'))
stemmer = PorterStemmer()

def get_tokens(text):
    lowers = text.lower()
    clean = regex.sub('',lowers)
    tokens=nltk.word_tokenize(clean)
    return [w for w in tokens if not w in stopset]

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#Count Stemmed Tokens
comments['tokens'] = comments.body.apply(lambda text: Counter(stem_tokens(get_tokens(text), stemmer)))

#Comment Length
comments['comment_length'] = comments.body.apply(len)

#Number of Words
comments['n_tokens'] = comments.tokens.apply(len)

#Comment Keywords
comments['keywords'] = comments.body.apply(newspaper.nlp.keywords)

print comments.columns

Index([u'author', u'body', u'body_html', u'controversiality', u'created',
       u'created_utc', u'distinguished', u'downs', u'edited', u'gilded', u'id',
       u'likes', u'link_id', u'name', u'num_reports', u'parent_id', u'replies',
       u'score', u'subreddit', u'ups', u'pid', u'tokens', u'comment_length',
       u'n_tokens', u'keywords'],
      dtype='object')


In [6]:
#Create Article Features
#########################
#Tokens
articles['tokens'] = articles['text'].apply(lambda text: Counter(stem_tokens(get_tokens(text), stemmer)))

#Article Length
articles['article_len'] = articles['text'].apply(len)

#Number of Words
articles['n_tokens'] = articles['text'].apply(len)

articles.columns

Index([u'authors', u'keywords', u'publish_date', u'summary', u'text', u'url',
       u'author', u'created_utc', u'domain', u'downs', u'gilded', u'is_self',
       u'likes', u'media', u'id', u'num_comments', u'num_reports', u'over_18',
       u'permalink', u'score', u'selftext', u'subreddit', u'thumbnail',
       u'title', u'ups', u'tokens', u'article_len', u'n_tokens'],
      dtype='object')

Lets write the final article and comment data to file.

In [76]:
articles.to_csv('./train_articles.csv',sep=',',index=False)
comments.to_csv('./train_comments.csv',sep=',',index=False)

Now we have raw content, stemmed tokens, keywords for each comment and article. Now we can start to look at the relationships between comment content and article content. We compouted the length to normalize some of the similarities. 

Should use weighted sim like weightign each word by its tf-idf 

In [7]:
import math
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [8]:
def keyword_sim(comment_row):
    try:
        c_kw = comment_row['keywords']
        a_kw = articles[articles['id']==comment_row['pid']]['keywords'].iloc[0]
        sim = get_cosine(c_kw,a_kw)
    except:
        sim = 0.0
    return sim

comments['keyword_sim']=comments.apply(keyword_sim,axis=1)

In [9]:
def token_sim(comment_row):
    try:
        c = comment_row['tokens']
        a = articles[articles['id']==comment_row['pid']]['tokens'].iloc[0]
        sim = get_cosine(c,a)
    except:
        sim = 0.0
    return sim

comments['token_sim']=comments.apply(token_sim,axis=1)    

In [10]:
#Here We Build a Vocab for each user
user_vocab = {}
for author,vocab in comments[['author','tokens']].groupby('author'):
    user_vocab[author] = sum((Counter(dict(x)) for x in vocab.tokens),Counter())

In [11]:
def user_vocab_tokens_sim(comment_row):
    try:
        c = user_vocab[comment_row.author]
        a = articles[articles['id']==comment_row['pid']]['tokens'].iloc[0]
        sim = get_cosine(c,a)
    except:
        sim = 0.0
    return sim

comments['user_vocab_tokens']=comments.apply(user_vocab_tokens_sim,axis=1)

In [22]:
def user_vocab_kw_sim(comment_row):
    try:
        c = user_vocab[comment_row.author]
        a = articles[articles['id']==comment_row['pid']]['keywords'].iloc[0]
        sim = get_cosine(c,a)
    except:
        sim = 0.0
    return sim

comments['user_vocab_kw']=comments.apply(user_vocab_kw_sim,axis=1)

In [23]:
sim_features = comments[['id','likes','link_id','name','parent_id','score','subreddit','ups','pid','comment_length','n_tokens','keyword_sim','token_sim','user_vocab_tokens','user_vocab_kw']]
sim_features.columns

Index([u'id', u'likes', u'link_id', u'name', u'parent_id', u'score',
       u'subreddit', u'ups', u'pid', u'comment_length', u'n_tokens',
       u'keyword_sim', u'token_sim', u'user_vocab_tokens', u'user_vocab_kw'],
      dtype='object')

In [25]:
sim_features.to_csv('./new_sim_features.csv',sep=',',index=False)

In [26]:
sim_features = pd.read_csv('./new_sim_features.csv')

In [27]:
sim_features.user_vocab_kw[90:100]

90    0.182951
91    0.162302
92    0.000000
93    0.000000
94    0.103394
95    0.166022
96    0.243600
97    0.000000
98    0.063660
99    0.013082
Name: user_vocab_kw, dtype: float64